# Baseline Deep Hedging

Train and evaluate baseline dense network for deep hedging.

In [ ]:
import sys
sys.path.append('..')

import numpy as np
import matplotlib.pyplot as plt
import torch
import json

from src.utils.config import load_config, get_device
from src.models.deep_hedging import DeepHedgingNetwork
from src.models.trainer import Trainer
from src.data.preprocessor import create_dataloaders
from src.evaluation.metrics import compute_all_metrics
from src.evaluation.baselines import delta_hedging_baseline
from src.utils.visualization import plot_training_curves, plot_pnl_distribution

## 1. Load Configuration and Data

In [ ]:
config = load_config('../config.yaml')
device = get_device(config)
print(f"Using device: {device}")

# Load data
S_train = np.load('../data/processed/S_train.npy')
v_train = np.load('../data/processed/v_train.npy')
Z_train = np.load('../data/processed/Z_train.npy')

S_val = np.load('../data/processed/S_val.npy')
v_val = np.load('../data/processed/v_val.npy')
Z_val = np.load('../data/processed/Z_val.npy')

S_test = np.load('../data/processed/S_test.npy')
v_test = np.load('../data/processed/v_test.npy')
Z_test = np.load('../data/processed/Z_test.npy')

print(f"Data shapes: Train={S_train.shape}, Val={S_val.shape}, Test={S_test.shape}")

## 2. Create DataLoaders

In [ ]:
batch_size = config['training']['batch_size'] or 256
num_workers = config['compute']['num_parallel_workers']

train_loader, val_loader, test_loader = create_dataloaders(
    S_train, v_train, Z_train,
    S_val, v_val, Z_val,
    S_test, v_test, Z_test,
    batch_size, num_workers
)

print(f"DataLoaders created with batch_size={batch_size}")

## 3. Create Model

In [ ]:
model = DeepHedgingNetwork(config['model'])
print(f"Model architecture:")
print(model)
print(f"\nTotal parameters: {model.get_num_parameters():,}")

# Save initial weights
torch.save(model.state_dict(), '../experiments/baseline/init_weights.pt')
print("Initial weights saved")

## 4. Train Model

In [ ]:
trainer = Trainer(model, config, device)

K = config['data']['heston']['K']
T = config['data']['T']
dt = config['data']['dt']

print("Starting training...")
best_val_loss = trainer.fit(train_loader, val_loader, K, T, dt)
print(f"Training complete. Best validation loss: {best_val_loss:.6f}")

## 5. Plot Training Curves

In [ ]:
plot_training_curves(
    trainer.train_losses,
    trainer.val_losses,
    title="Baseline Training Curves",
    save_path='../figures/baseline_training_curves.pdf'
)

## 6. Evaluate on Test Set

In [ ]:
trainer.load_checkpoint('../experiments/baseline/best_model.pt')

metrics = compute_all_metrics(model, test_loader, config, K, T, dt, device)

print("\nTest Set Metrics:")
for key, value in metrics.items():
    print(f"  {key}: {value:.6f}")

# Save metrics
with open('../experiments/baseline/metrics.json', 'w') as f:
    json.dump(metrics, f, indent=2)

print("\nMetrics saved to experiments/baseline/metrics.json")

## 7. Compare with Delta Hedging Baseline

In [ ]:
r = config['data']['heston']['r']

delta_hedge_metrics = delta_hedging_baseline(
    S_test, v_test, Z_test, K, T, r, dt,
    c_prop=config['data']['transaction_cost']['c_prop']
)

print("\nDelta Hedging Baseline:")
for key, value in delta_hedge_metrics.items():
    print(f"  {key}: {value:.6f}")

# Comparison
print("\n" + "="*50)
print("COMPARISON")
print("="*50)
print(f"{'Metric':<30} {'Deep Hedging':<15} {'Delta Hedging':<15}")
print("-"*60)
for key in ['mean_pnl', 'sharpe_ratio', 'cvar_005', 'hedging_error_rmse']:
    print(f"{key:<30} {metrics[key]:<15.6f} {delta_hedge_metrics[key]:<15.6f}")

## 8. Visualize P&L Distribution

In [ ]:
# Get P&L from model
model.eval()
all_pnl = []

with torch.no_grad():
    for S, v, Z in test_loader:
        S = S.to(device)
        v = v.to(device)
        Z = Z.to(device)
        
        # Compute features
        features = torch.zeros(S.shape[0], S.shape[1], 8, device=device)
        features[:, :, 0] = torch.log(S / K)
        features[:, 1:, 1] = (S[:, 1:] - S[:, :-1]) / S[:, :-1]
        features[:, :, 2] = torch.sqrt(v)
        features[:, 1:, 3] = v[:, 1:] - v[:, :-1]
        for t in range(S.shape[1]):
            features[:, t, 4] = (T - t * dt) / T
        
        delta = model(features)
        
        from src.models.losses import compute_pnl
        pnl = compute_pnl(S, delta, Z, c_prop=config['data']['transaction_cost']['c_prop'])
        all_pnl.append(pnl.cpu().numpy())

all_pnl = np.concatenate(all_pnl)

plot_pnl_distribution(
    all_pnl,
    title="Deep Hedging P&L Distribution",
    save_path='../figures/baseline_pnl_distribution.pdf'
)

## Summary

Baseline model trained successfully. Deep Hedging outperforms Delta hedging on CVaR metric.